In [15]:
import json
from datetime import datetime
from datetime import date
from pycoingecko import CoinGeckoAPI
import pandas as pd

In [61]:
today = date.today()
#retrieving price from 2020/06/01 till now
days = pd.date_range(start="2021-05-27",end=today.strftime("%Y-%m-%d"))

In [63]:
cg = CoinGeckoAPI()
dogePrice = []
for day in days:
    day = day.strftime("%d-%m-%Y")
    price = cg.get_coin_history_by_id(id='dogecoin',date=day, localization='false')
    for e in ["_id", "symbol", "name", "image", "developer_data", "commit_count_4_weeks", "public_interest_stats"]:
        price.pop(e, None)
    price["Date"] = datetime.strptime(day, "%d-%m-%Y")
    dogePrice.append(price)

In [64]:
for i in range(len(dogePrice)):
    dogePrice[i]["variation_next24h"] = 0
    if(i<len(dogePrice)-1):
        B = dogePrice[i+1]["market_data"]["current_price"]["usd"]
        A = dogePrice[i]["market_data"]["current_price"]["usd"]
        dogePrice[i]["variation_next24h"] =  ((B - A)/A )* 100

In [65]:
from pymongo import MongoClient 
myclient = MongoClient("mongodb://localhost:27017/") 
db = myclient["DataMan"]
Collection = db["DogeHistorical"]
if isinstance(dogePrice, list):
    Collection.insert_many(dogePrice)  
else:
    Collection.insert_one(dogePrice)
myclient.close()